In [1]:
# Imports

import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import datetime,time,warnings,itertools,sys,os,math,platform

import pandas as pd

In [2]:
# list rL = flatten_list(list L)
# Function which takes in an arbitrarily nested list and returns a flattened version of it
def flatten_list(L):
    rL = []
    for item in L:
        if type(item) is list:
            itemL = flatten_list(item)
        else:
            itemL = [item]
        for x in itemL:
            rL.append(x)
    return rL

In [27]:
# System-dependent configuration of read path

if platform.system()=='Darwin':
    path = './Users/janlucasdeinhard/Dropbox/PhD Prep/SHS DQ/'
else:
    path = '..\\..\\TF_data\\Dropbox\\PhD Prep\\SHS DQ\\'
path += 'CompetencyQueries.sql'

In [29]:
# Read in SQL file
with open(path,'r') as fd: queries = fd.read()

In [30]:
# Preprocessing

# Map out the known problems
queries = queries.replace('DATAITEM','DATA_ITEMS')
queries = queries.replace('TECHNICAL_SYSTEMS_STATUS','TECHNICAL_SYSTEM_STATUS')
queries = queries.replace('DQ_METRIC_INSTANCES','DQ_METRIC_INSTANCE')
queries = queries.replace('ITEMSS','ITEMS')

# List of queries
qlist = [k[k.find('SELECT'):] for k in queries.split('\n\n')]
# Flattened list of all lines
f_qlist = flatten_list([q.split('\n') for q in qlist])

In [31]:
# List of all keywords which will be targeted
keywords = [
    'SELECT',
    'FROM',
    'WHERE',
    'JOIN',
    'GROUP BY',
    'ORDER BY',
    ' ON '
]

# Dictionary to map all keywords to what subsequent data structure it points to
statement_pointer_type = {
    'SELECT':'COLUMN',
    'FROM':'TABLE',
    'WHERE':'COLUMN',
    'JOIN':'TABLE',
    'GROUP BY':'COLUMN',
    'ORDER BY':'COLUMN',
    ' ON ':'COLUMN'
}

In [32]:
# Iterate all queries

# Final data frame
df = pd.DataFrame()

# Iterate all queries and decompose
ctr = 0
for q in qlist:
    try:
        q = q.replace('LEFT JOIN','JOIN').replace('RIGHT JOIN','JOIN').replace('FULL OUTER JOIN','JOIN').replace('INNER JOIN','JOIN')
        #q = q.replace('\n','').replace('\t','')
        # Create dataframe containing all start positions of keywords
        tf = pd.DataFrame([(k,q.find(k)) for k in keywords if q.find(k)!=-1]).sort_values(by=1,ascending=True).rename(columns={
            0:'Keyword',
            1:'Start Position'
        })
        # Determine end position of target string as start position of next keyword
        tf['End Position'] = tf['Start Position'].shift(-1).fillna(-1).astype(int)
        # Cut out string
        tf['Value'] = tf[['Keyword','Start Position','End Position']].apply(lambda x: 
            q[x['Start Position']:x['End Position']].replace(x['Keyword'],'').strip().replace('\n','').replace('\t','')
        ,axis=1)
        # Drop unnecessary columns
        tf = tf.drop(columns=['Start Position','End Position'])
        # Attach query ID
        tf['QueryID'] = ctr
        tf['Original Query'] = q
        # Attach to full data frame
        df = pd.concat([df,tf],axis=0)
    except:
        continue    
    # Count up
    ctr += 1
# Create index
df = df.reset_index(drop=True)

In [33]:
# Post-processing

# Attach Value Type
df['Value Type'] = df['Keyword'].map(statement_pointer_type)
# Identify sub-queries
df['Value Type'] = df[['Keyword','Value','Value Type']].apply(lambda x: 'SUBQUERY' if x['Keyword']=='FROM' and 'SELECT' in x['Value'] else x['Value Type'],axis=1)
# Drop table names
df['Value'] = df[['Value','Value Type']].apply(lambda x: x['Value'].split(' ')[0] if x['Value Type']=='TABLE' else x['Value'],axis=1)

In [34]:
df[df['Value Type']=='TABLE']['Value'].drop_duplicates()

1                        MATCH_TABLE
10                     DQ_VIOLATIONS
11                DQ_METRIC_INSTANCE
15    DATA_ITEMS_TO_TECHNICAL_SYSTEM
16                 TECHNICAL_SYSTEMS
21                        DATA_ITEMS
26    DATA_ITEMS_TECHNICAL_REFERENCE
29       METADATA.VALIDATION_QUERIES
34           TECHNICAL_SYSTEM_STATUS
45     TECHNICAL_METADATA_DATA_ITEMS
68         TECHNICAL_SYSTEMS_PROBLEM
71              DATA_ITEMS_TO_TABELS
72                   METADATA.TABLES
76         INTERNAL_REFERENCE_POINTS
79         EXTERNAL_REFERENCE_POINTS
82      DQ_AGGREGATE_SCORE_PER_SCOPE
85         DQ_METRIC_INSTANCE_RESULT
90                         DQ_METRIC
Name: Value, dtype: object

In [38]:
df[df['Value Type']=='COLUMN']#['Value'].drop_duplicates()

,Keyword,Value,QueryID,Original Query,Value Type
0,SELECT,"SUM(""Matched Instances Count"")",0,"SELECT SUM(""Matched Instances Count"")\nFROM MA...",COLUMN
2,WHERE,"""Entity""='Contacts'AND ((""System A"" = 'ELOQUA'...",0,"SELECT SUM(""Matched Instances Count"")\nFROM MA...",COLUMN
3,SELECT,"DISTINCT ""Matching Method""",1,"SELECT DISTINCT ""Matching Method""\nFROM MATCH_...",COLUMN
5,WHERE,"""Entity""='Contacts",1,"SELECT DISTINCT ""Matching Method""\nFROM MATCH_...",COLUMN
6,SELECT,"""Matching Method"",SUM(""Matched Instances Count"")",2,"SELECT ""Matching Method"",SUM(""Matched Instance...",COLUMN
...,...,...,...,...,...
110,WHERE,"""Deactivation_Date"" = '9999-12-31 00:00:00'AND...",29,"SELECT ""Name"",""DQ Constraint Origin""\nFROM DQ_...",COLUMN
111,SELECT,"""Segmentation Type"",""Segment"",SUM(""Matched Ins...",30,"SELECT \n\t""Segmentation Type"",\n\t""Segment"",\...",COLUMN
113,WHERE,"""Entity""='Contacts'AND ""Segmentation Type"" IN ...",30,"SELECT \n\t""Segmentation Type"",\n\t""Segment"",\...",COLUMN
114,GROUP BY,"""Segmentation Type"",""Segment""",30,"SELECT \n\t""Segmentation Type"",\n\t""Segment"",\...",COLUMN
